In [ ]:
import os

from astropy.time import Time
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', logo=False)
import numpy as np
import pandas as pd

from lsst_efd_client import EfdClient

In [ ]:
location = "summit_efd"
efd = EfdClient(location)

In [ ]:
start = Time('2020-03-16 20:00:00', scale='tai')
end = Time('2020-03-17 07:00:00', scale='tai')

In [ ]:
atcamera_df = await efd.select_time_series('lsst.sal.ATCamera.logevent_endReadout',
                                           ['private_sndStamp', 'imageName', 'timeStamp'],
                                           start, end)
#print(atcamera_df)

In [ ]:
atarchiver_df = await efd.select_time_series('lsst.sal.ATArchiver.logevent_imageInOODS',
                                             ['private_sndStamp', 'statusCode', 'description'],
                                             start, end)
#print(atarchiver_df)

In [ ]:
aths_df = await efd.select_time_series('lsst.sal.ATHeaderService.logevent_largeFileObjectAvailable',
                                       ['private_sndStamp', 'url', 'generator'],
                                       start, end)
#print(aths_df)

In [ ]:
atcamera_send_time = []
atarchiver_send_time = []
atarchiver_arrival_time = []
aths_send_time = []
for row in atcamera_df.itertuples():
    atcamera_time = row.private_sndStamp
    atcamera_image = row.imageName
    index = atarchiver_df['description'].str.contains(f"{atcamera_image}", na=False)
    try:
        if atarchiver_df['statusCode'][index].values[0] == 0.0:
            atarchiver_time = atarchiver_df['private_sndStamp'][index].values[0]
            have_archiver_file = True
        else:
            have_archiver_file = False
    except IndexError:
        have_archiver_file = False
    index = aths_df['url'].str.contains(f"{atcamera_image}", na=False)
    try:
        aths_time = aths_df['private_sndStamp'][index].values[0]
        have_header_file = True
    except IndexError:
        have_header_file = False
    
    if have_header_file and have_archiver_file:
        atcamera_send_time.append(atcamera_time)
        atarchiver_send_time.append(atarchiver_time)
        aths_send_time.append(aths_time)
        atarchiver_arrival_time.append(atarchiver_time - atcamera_time)

In [ ]:
atcamera_send_time_array = np.array(atcamera_send_time)
atarchiver_send_time_array = np.array(atarchiver_send_time)
atarchiver_arrival_time_array = np.array(atarchiver_arrival_time)
aths_send_time_array = np.array(aths_send_time)
time_diff_str = 'T^ATA_IO_Send - T^ATC_EOR_Send'

In [ ]:
hist = np.histogram(atarchiver_arrival_time_array, bins=np.arange(start=0, stop=100, step=2))
aat = hv.Histogram(hist, kdims=hv.Dimension('time differences', label=time_diff_str, unit='s'),
                   label='Distribution of 𝚫T')

In [ ]:
cst_vs_aat = hv.Scatter((atcamera_send_time_array.astype('datetime64[s]'), atarchiver_arrival_time_array),
                        kdims='T^ATC_EOR_Send', vdims=hv.Dimension('time difference', label=time_diff_str, unit='s'),
                        label='𝚫T During Image Taking')

In [ ]:
delta_t1 = (aths_send_time_array - atcamera_send_time_array)
min_x = np.min(delta_t1)
max_x = np.max(delta_t1)
hist = np.histogram(delta_t1, bins=np.linspace(min_x, max_x, 50))
ahst = hv.Histogram(hist, kdims=hv.Dimension('time differences', 
                                             label='T^ATHS_LFOA_Send - T^ATC_EOR_Send', unit='s'),
                    label='Distribution of 𝚫T1')

In [ ]:
#ahst + est + hfst
ahst

In [ ]:
%%opts Scatter [padding=(0.05, 0.05)]
#hv.renderer('bokeh').theme = None
aat + cst_vs_aat

In [ ]:
indexes = np.where(atarchiver_arrival_time_array > 40.0)
for c, a in zip(atarchiver_arrival_time_array[indexes], atcamera_send_time_array.astype('datetime64[s]')[indexes]):
    print(f"{c:.2f}\t{a}")